### 0. Helper functions

In [1]:
import numpy as np

class bin_data:
    def __init__(self, env):
        self.total_capacity = np.full(env.num_bins, 0) # Total capacity seen across all trials
        self.capacity_used = np.full(env.num_bins, 0) # Total capacity used across all trials
        self.steps_taken = 0 # Number of timesteps taken before trial terminated
        self.placed = 0 # Number of items correctly placed
        self.misplaced = 0
        self.discarded = 0
        self.data_points = 0 # Number of trials we've witnessed
    

    def log(self, env):
        self.total_capacity = np.add(self.total_capacity, env.capacity)
        self.capacity_used = np.add(self.capacity_used, env.state[:-1])
        self.steps_taken += sum(env.logs.values())
        self.placed += env.logs["placed"]
        self.misplaced += env.logs["misplaced"]
        self.discarded += env.logs["discarded"]
        self.data_points += 1
        

    def get_avg(self):
        percentages = 100 - ((env.state/env.capacity) * 100)
        
        return {
            "steps" : self.steps_taken / self.data_points,
            "utilization" : sum(percentages) / len(percentages),
            "accuracy" : (self.placed / (self.placed + self.misplaced)) * 100
        }
    

    def print_data(self):
        data = self.get_avg()
        print(f'Average number of steps taken: {data["steps"]}')
        print(f'Average bin utilization: {round(data["utilization"], 2)}%')
        print(f'Accuracy: {round(data["accuracy"], 2)}%')

### 1. Import dependencies

In [ ]:
!pip install tensorflow==2.7.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [2]:
from BinPackingEnvironment1D import BinPacking

### 2. Create environment

In [3]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [4]:
env.action_space.sample()

1

In [5]:
env.observation_space.sample()

array([17, 14,  8, 13, 18,  3,  2,  6, 12,  3, 13])

### 3. Run baseline test (No ML)

In [6]:
control_data = bin_data(env)

In [7]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    control_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-562
{'placed': 40, 'misplaced': 545, 'discarded': 57}
Episode:2 Score:-172
{'placed': 41, 'misplaced': 188, 'discarded': 25}
Episode:3 Score:-389
{'placed': 46, 'misplaced': 397, 'discarded': 38}
Episode:4 Score:-162
{'placed': 39, 'misplaced': 177, 'discarded': 24}
Episode:5 Score:-922
{'placed': 39, 'misplaced': 880, 'discarded': 81}
Episode:6 Score:-548
{'placed': 50, 'misplaced': 536, 'discarded': 62}
Episode:7 Score:-750
{'placed': 50, 'misplaced': 725, 'discarded': 75}
Episode:8 Score:-746
{'placed': 38, 'misplaced': 721, 'discarded': 63}
Episode:9 Score:-128
{'placed': 42, 'misplaced': 160, 'discarded': 10}
Episode:10 Score:-333
{'placed': 42, 'misplaced': 343, 'discarded': 32}


### 4. Train an RL Model

In [8]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [9]:
# Will throw an error if these don't exist
log_path = os.path.join('Training', 'Logs')

In [10]:
env = BinPacking(num_bins=10, capacity=20, min_item_size=1)

In [11]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [12]:
model.learn(total_timesteps=80000)

Logging to Training\Logs\PPO_53
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 334      |
|    ep_rew_mean     | -249     |
| time/              |          |
|    fps             | 295      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 362          |
|    ep_rew_mean          | -273         |
| time/                   |              |
|    fps                  | 413          |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0082404995 |
|    clip_fraction        | 0.0594       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.39        |
|    explained_variance   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 274         |
|    ep_rew_mean          | -183        |
| time/                   |             |
|    fps                  | 610         |
|    iterations           | 11          |
|    time_elapsed         | 36          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.011472365 |
|    clip_fraction        | 0.0953      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.18       |
|    explained_variance   | 0.641       |
|    learning_rate        | 0.0003      |
|    loss                 | 31.6        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0163     |
|    value_loss           | 55.8        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 267   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 135         |
|    ep_rew_mean          | -41.3       |
| time/                   |             |
|    fps                  | 595         |
|    iterations           | 21          |
|    time_elapsed         | 72          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.010742598 |
|    clip_fraction        | 0.108       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.75       |
|    explained_variance   | 0.637       |
|    learning_rate        | 0.0003      |
|    loss                 | 10.9        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.0179     |
|    value_loss           | 29.8        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 133   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 105         |
|    ep_rew_mean          | -11.7       |
| time/                   |             |
|    fps                  | 593         |
|    iterations           | 31          |
|    time_elapsed         | 107         |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.010464398 |
|    clip_fraction        | 0.123       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.48       |
|    explained_variance   | 0.738       |
|    learning_rate        | 0.0003      |
|    loss                 | 5.73        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0247     |
|    value_loss           | 18.1        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 104   

### 5. Save model

In [13]:
#PPO_Path = os.path.join('Training', 'Saved Models', 'Constant_PPO_Model_Discard_Penalty')

In [14]:
#model.save(PPO_Path)

### 6. Load model

In [15]:
#model = PPO.load(PPO_Path, env=env)

### 7. Test model

In [16]:
real_data = bin_data(env)
env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

In [17]:
MAX_STEPS = 1000
episodes = 10

for episode in range(1, episodes+1):
    obs = env.reset()
    steps = 0
    done = False
    score = 0
    
    while not done and steps < MAX_STEPS:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        steps += 1

    print('Episode:{} Score:{}'.format(episode,score))
    print(env.logs)
    real_data.log(env)
    env.logs = { 'placed':0, 'misplaced':0, 'discarded':0 }

Episode:1 Score:-12
{'placed': 54, 'misplaced': 19, 'discarded': 47}
Episode:2 Score:16
{'placed': 46, 'misplaced': 9, 'discarded': 21}
Episode:3 Score:-13
{'placed': 45, 'misplaced': 20, 'discarded': 38}
Episode:4 Score:11
{'placed': 47, 'misplaced': 16, 'discarded': 20}
Episode:5 Score:24
{'placed': 45, 'misplaced': 7, 'discarded': 14}
Episode:6 Score:-4
{'placed': 46, 'misplaced': 19, 'discarded': 31}
Episode:7 Score:-1
{'placed': 45, 'misplaced': 9, 'discarded': 37}
Episode:8 Score:11
{'placed': 39, 'misplaced': 6, 'discarded': 22}
Episode:9 Score:-4
{'placed': 44, 'misplaced': 19, 'discarded': 29}
Episode:10 Score:-7
{'placed': 44, 'misplaced': 19, 'discarded': 32}


In [18]:
control_data.print_data()

Average number of steps taken: 556.6
Average bin utilization: 97.73%
Accuracy: 8.37%


In [19]:
real_data.print_data()

Average number of steps taken: 88.9
Average bin utilization: 97.73%
Accuracy: 76.09%
